In [11]:
import requests
import jsonlines
import pandas as pd
from requests_oauthlib import OAuth1

**PART 1**

In [2]:
API_KEY = 'enter_api_key'
API_SECRET = 'enter_api_secret_key'
USER_OAUTH_TOKEN = 'enter_access_token'
USER_OAUTH_TOKEN_SECRET = 'enter_access_token_secret'
SCREEN_NAME = 'midasIIITD'

In [3]:
def get_user_tweets(handle, auth):
    final_data = []
    url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
    params = {
        'screen_name': handle, 
        'count': 200 #Setting the count to maximum number of tweets that can be retrieved at a time
    }
    while 1:
        R = requests.get(url, auth=auth, params=params)
        data = R.json()
        final_data += data
        length  = len(data)
        if length<200:
            break
        max_id = data[199]['id']-1
        params['max_id'] = max_id
    return final_data 

**Fetch all the tweets done by midas@IIITD twitter handle and dump the responses into JSONlines file**

In [21]:
auth =  OAuth1(API_KEY, API_SECRET, USER_OAUTH_TOKEN, USER_OAUTH_TOKEN_SECRET)
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
#checking the validity of user credentials
print(requests.get(url, auth=auth))
#get tweets 
data = get_user_tweets(SCREEN_NAME, auth)
#Dump the data into JSONlines file
with jsonlines.open('midas_tweets.jsonl', mode='w') as writer:
    writer.write(data)

<Response [200]>


**PART 2**

In [22]:
def get_images_count(data):
    tweet_counts = len(data)
    images_count = []
    for i in range(tweet_counts):
        try:
            count = 0
            x = len(info[i]['extended_entities']['media'])
            for j in range(x):
                if info[i]['extended_entities']['media'][j]['type']=='photo':
                    count += 1
            images_count.append(count)
        except:
            images_count.append('None')
            continue
    return images_count

In [24]:
#Read the data from jsonlines files
with jsonlines.open('midas_tweets.jsonl') as reader:
    for obj in reader:
        data = obj

images_count = pd.Series(get_images_count(data))
df = pd.DataFrame(data)
df['images_count'] = images_count.values
df = df[['created_at','text','favorite_count','retweet_count','images_count']]
df.rename(columns={'created_at':'data_and_time'}, inplace=True)
df

,data_and_time,text,favorite_count,retweet_count,images_count
0,Tue Mar 26 05:54:49 +0000 2019,@IEEEBigMM19 @ACMMM19 and 6 days left for work...,1,0,None
1,Tue Mar 26 05:50:10 +0000 2019,RT @IEEEBigMM19: Hurry Up!\n6 Days left for Ab...,0,3,None
2,Mon Mar 25 13:01:57 +0000 2019,Congratulations @midasIIITD students Simra Sha...,14,1,None
3,Sun Mar 24 18:44:01 +0000 2019,The last date for submitting a solution for th...,8,3,None
4,Sun Mar 24 18:26:02 +0000 2019,RT @IIITDelhi: @IIITDelhi invites application ...,0,4,None
5,Sun Mar 24 11:34:27 +0000 2019,One more week is left to submit the workshop p...,4,0,None
6,Sun Mar 24 06:23:37 +0000 2019,RT @IEEEBigMM19: We are honored to have Dr. Ch...,0,5,None
7,Sun Mar 24 06:23:14 +0000 2019,RT @IEEEBigMM19: Distinguished researchers Dr....,0,3,None
8,Wed Mar 20 08:19:24 +0000 2019,@IEEEBigMM19 is also available on Facebook now...,1,1,None
9,Wed Mar 20 02:40:07 +0000 2019,RT @IEEEBigMM19: BigMM 2019 : IEEE BigMM 2019 ...,0,5,None
